# Visualize results on the flattened brain
Use connectome-workbench to visualize the results on the flatten brain

## conda install
- cifti and dlabel
- conda install -c bioconda connectome-workbench

## An example to change dlabel for visulization  

### check data and afni atlas template

In [ ]:
import numpy as np
import os
import nibabel as nib
import copy

project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/spatial_process"
fMRI_data_dir="/media/statespace/Spatial/sptialworkspace/spatialnavigation/fMRI_data/spatialfMRI_nii"
preprocessed_dir=project_dir+"/preprocess/preprocessed/afni_2021"
processed_dir=project_dir+"/process/processed"
score_dir_out = project_dir+"/process/processed/sub-mni/"

# read afni atlas MNI_Glasser_HCP_v1.0
file_name_str = 'MNI_Glasser_HCP_v1.0'
score_map_path = os.path.join(score_dir_out, file_name_str+'.nii.gz')
image_file = nib.load(score_map_path)
print("MNI_Glasser_HCP_v1 shape:", image_file.get_fdata().shape)

image_data = image_file.get_fdata()
voxel_num = np.count_nonzero(image_data[image_data!=0])
print("MNI_Glasser_HCP_v1 non zero num:", voxel_num)

# read afni atlas MNI_Glasser_HCP_v1.0
score_map_path = os.path.join(score_dir_out, 'Q1-Q6_RelatedValidation210.CorticalAreas_dil_Final_Final_Areas_Group_Colors_with_Atlas_ROIs2.32k_fs_LR.dlabel.nii')
image_file = nib.load(score_map_path)
print("wb scene shape:", image_file.shape)


### get data and header


In [ ]:
cifti_data = image_file.get_fdata(dtype=np.float32)
cifti_hdr = image_file.header
nifti_hdr = image_file.nifti_header

print('cifti_data:', cifti_data.shape)

# axes
cifti_axes = [cifti_hdr.get_axis(i) for i in range(image_file.ndim)]
# print('cifti_axes:',cifti_axes)
# print(list(cifti_axes[0]))
# print(list(cifti_axes[1].iter_structures()))

regions_labels = cifti_axes[0].get_element(0)[1]
print(len(regions_labels))
# print(regions_labels)
for i in range(380):
  print(i, ':', regions_labels[i])


### change the label of axis


In [ ]:
# load score
from scipy.io import loadmat

project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/spatial_process"
file_dir = project_dir+"/process/shared_regions_modeling/temp_data/"

file_str = "vitae_pca_explainable_variance_ratio_middlelayers_brain_regions"

matdic = loadmat(file_dir+file_str+".mat")

sub_score = matdic["sub_score"]
print("sub_score shape =", sub_score.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

temp_cmap = cm.get_cmap('coolwarm')
print(temp_cmap)

temp_sub_score = np.squeeze(sub_score)
temp_sub_score_color_norm = (temp_sub_score - np.min(temp_sub_score)) / (np.max(temp_sub_score) - np.min(temp_sub_score))
coolwarm_colors = temp_cmap(temp_sub_score_color_norm)


In [ ]:
import copy

cifti_label_axis = copy.deepcopy(cifti_axes[0])
cifti_brain_model_axis = cifti_axes[1]

for i_region in range(0, 180):

  temp_orig_tuple = cifti_label_axis[0][1][i_region + 1]
  
  cifti_label_axis[0][1][i_region + 1] = tuple((temp_orig_tuple[0], tuple(coolwarm_colors[i_region])))
  # cifti_label_axis[0][1][i_region + 180 + 1] = tuple((temp_orig_tuple[0], tuple(coolwarm_colors[i_region])))

cifti_new_hdr = nib.cifti2.cifti2.Cifti2Header()
temp_hdr = cifti_new_hdr.from_axes(tuple((cifti_label_axis,cifti_brain_model_axis)))


In [ ]:
# write the new cifti file
new_img_file = nib.cifti2.cifti2.Cifti2Image(cifti_data, header=temp_hdr, nifti_header=nifti_hdr)
temp_cifti_hdr = new_img_file.header
temp_cifti_axes = [temp_cifti_hdr.get_axis(i) for i in range(new_img_file.ndim)]
temp_cifti_label_axis = temp_cifti_axes[0]
# print(temp_cifti_label_axis[0][1]) # label dict


In [ ]:
# show the label value
regions_labels = temp_cifti_label_axis.get_element(0)[1]
print(len(regions_labels))
# print(regions_labels)
# for i in range(380):
#   print(i, ':', regions_labels[i])


### save the data

In [ ]:
score_map_path = os.path.join(score_dir_out, file_str+'.32k_fs_LR.dlabel.nii')
new_img_file.to_filename(score_map_path)

## Export data to connectome workbench

### environment setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy.io import loadmat

project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/spatial_process"
file_dir = project_dir+"/process/shared_region_dims/temp_data/"

num_total_dims = 1409

### road types explainable variance score

In [ ]:
temp_cmap = cm.get_cmap('RdBu_r')
print(temp_cmap)

file_str_list = []

# file_str = "vit_"+ str(num_total_dims) + "_dims"+ "_geometry_ev_score_brain_regions"
# file_str_list.append(file_str)

file_str = "vit_"+ str(num_total_dims) + "_dims"+ "_roadtypes_ev_score_brain_regions"
file_str_list.append(file_str)

matdic = loadmat(file_dir+file_str+".mat")
sub_score = matdic["sub_score"]
pvals_corrected = matdic["pvals_corrected"]

p_value_threshold = 0.05
print("number of effective regions:", np.sum(pvals_corrected < p_value_threshold))

explained_variance_type_list = ["raw", "binary"]
for i_explained_variance_type in explained_variance_type_list:
  
  temp_value_sub = copy.deepcopy(sub_score)

  if i_explained_variance_type == "binary":
    temp_value_sub[pvals_corrected < p_value_threshold] = np.max(temp_value_sub)
    temp_value_sub[pvals_corrected >= p_value_threshold] = np.min(temp_value_sub)
  elif i_explained_variance_type == "raw":
    temp_value_sub[pvals_corrected >= p_value_threshold] = np.min(temp_value_sub)

  temp_sub_score = np.squeeze(temp_value_sub)

  temp_sub_score_min = np.min(temp_sub_score)
  temp_sub_score_max = np.max(temp_sub_score)

  print("sub_score_min:", temp_sub_score_min)
  print("sub_score_max:", temp_sub_score_max)

  temp_sub_score_color_norm = (temp_sub_score - temp_sub_score_min) / (temp_sub_score_max - temp_sub_score_min)
  coolwarm_colors = temp_cmap(temp_sub_score_color_norm)

  # find cifti label axis
  cifti_label_axis = copy.deepcopy(cifti_axes[0])
  cifti_brain_model_axis = cifti_axes[1]

  for i_region in range(0, 180):

    temp_orig_tuple = cifti_label_axis[0][1][i_region + 1]
    
    cifti_label_axis[0][1][i_region + 1] = tuple((temp_orig_tuple[0], tuple(coolwarm_colors[i_region])))

  cifti_new_hdr = nib.cifti2.cifti2.Cifti2Header()
  temp_hdr = cifti_new_hdr.from_axes(tuple((cifti_label_axis,cifti_brain_model_axis)))

  # create new image file
  new_img_file = nib.cifti2.cifti2.Cifti2Image(cifti_data, header=temp_hdr, nifti_header=nifti_hdr)

  score_map_path = os.path.join(score_dir_out, i_explained_variance_type + "_" + file_str+'.32k_fs_LR.dlabel.nii')
  new_img_file.to_filename(score_map_path)

### vit latent explainable variance score

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy.io import loadmat

project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/spatial_process"
file_dir = project_dir+"/process/shared_region_dims/temp_data/"

temp_cmap = cm.get_cmap('coolwarm')
print(temp_cmap)

file_str_list = []

file_str = "vit_"+ str(num_total_dims) + "_dims"+ "_midlayer_geometry_ev_score_brain_regions"
file_str_list.append(file_str)

# file_str = "vit_semantic_ev_score_brain_regions"
# file_str_list.append(file_str)

matdic = loadmat(file_dir+file_str+".mat")
sub_score = matdic["sub_score"]
pvals_corrected = matdic["pvals_corrected"]

p_value_threshold = 0.05
print("number of effective regions:", np.sum(pvals_corrected < p_value_threshold))

explained_variance_type_list = ["raw", "binary"]
for i_explained_variance_type in explained_variance_type_list:

  temp_value_sub = copy.deepcopy(sub_score)

  if i_explained_variance_type == "binary":
    temp_value_sub[pvals_corrected < p_value_threshold] = np.max(temp_value_sub)
    temp_value_sub[pvals_corrected >= p_value_threshold] = np.min(temp_value_sub)
  elif i_explained_variance_type == "raw":
    temp_value_sub[pvals_corrected >= p_value_threshold] = np.min(temp_value_sub)

  temp_sub_score = np.squeeze(temp_value_sub)

  temp_sub_score_min = np.min(temp_sub_score)
  temp_sub_score_max = np.max(temp_sub_score)

  print("sub_score_min:", temp_sub_score_min)
  print("sub_score_max:", temp_sub_score_max)

  temp_sub_score_color_norm = (temp_sub_score - temp_sub_score_min) / (temp_sub_score_max - temp_sub_score_min)
  coolwarm_colors = temp_cmap(temp_sub_score_color_norm)

  # find cifti label axis
  cifti_label_axis = copy.deepcopy(cifti_axes[0])
  cifti_brain_model_axis = cifti_axes[1]

  for i_region in range(0, 180):

    temp_orig_tuple = cifti_label_axis[0][1][i_region + 1]
    
    cifti_label_axis[0][1][i_region + 1] = tuple((temp_orig_tuple[0], tuple(coolwarm_colors[i_region])))

  cifti_new_hdr = nib.cifti2.cifti2.Cifti2Header()
  temp_hdr = cifti_new_hdr.from_axes(tuple((cifti_label_axis,cifti_brain_model_axis)))

  # create new image file
  new_img_file = nib.cifti2.cifti2.Cifti2Image(cifti_data, header=temp_hdr, nifti_header=nifti_hdr)

  score_map_path = os.path.join(score_dir_out, i_explained_variance_type + "_" + file_str+'.32k_fs_LR.dlabel.nii')
  new_img_file.to_filename(score_map_path)

#### number of effective regions vit midlayer explainable variance score

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy.io import loadmat

project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/spatial_process"
file_dir=project_dir+"/process/shared_region_dims/temp_data/"

temp_cmap = cm.get_cmap('coolwarm')
print(temp_cmap)

p_value_threshold = 0.05

# load all number of dims
from scipy.io import loadmat

file_dir = project_dir+"/process/shared_region_dims/temp_data/"
matdic = loadmat(file_dir + "vit_dims_list_semgeo_shared_region_dims_list_optimal_hard_threshold.mat")
num_total_dims_list = list(np.squeeze(matdic["num_total_dims_array"]))

bool_pvals_list = []
num_significant_regions_list = []
for i_num_total_dims in num_total_dims_list:

  num_total_dims = i_num_total_dims

  file_str = "vit_"+ str(num_total_dims) + "_dims"+ "_midlayer_geometry_ev_score_brain_regions"

  matdic = loadmat(file_dir+file_str+".mat")
  sub_score = np.squeeze(matdic["sub_score"])
  pvals_corrected = np.squeeze(matdic["pvals_corrected"])

  temp_value_sub = copy.deepcopy(sub_score)
  
  bool_temp = np.array(pvals_corrected < p_value_threshold)

  bool_pvals_list.append(bool_temp)

  num_significant_regions_list.append(np.sum(pvals_corrected < p_value_threshold))

  print(num_total_dims, "number of effective regions (pvals):", np.sum(pvals_corrected < p_value_threshold))
  temp_value_sub[pvals_corrected >= p_value_threshold] = np.min(temp_value_sub)

# find max
index_max = num_significant_regions_list.index(max(num_significant_regions_list))
print("index_max:", index_max, "num of regions:", num_significant_regions_list[index_max])
print("num_total_dims:", num_total_dims_list[index_max])

# show max
num_total_dims = num_total_dims_list[index_max]

file_str = "vit_"+ str(num_total_dims) + "_dims"+ "_midlayer_geometry_ev_score_brain_regions"

# load mat
matdic = loadmat(file_dir+file_str+".mat")
sub_score = np.squeeze(matdic["sub_score"])
pvals_corrected = np.squeeze(matdic["pvals_corrected"])

temp_value_sub = copy.deepcopy(sub_score)

# save for wb
explained_variance_type_list = ["raw", "binary"]

for i_explained_variance_type in explained_variance_type_list:

  if i_explained_variance_type == "binary":
    temp_value_sub[pvals_corrected < p_value_threshold] = np.max(temp_value_sub)
    temp_value_sub[pvals_corrected >= p_value_threshold] = np.min(temp_value_sub)
  elif i_explained_variance_type == "raw":
    temp_value_sub[pvals_corrected >= p_value_threshold] = np.min(temp_value_sub)

  temp_sub_score = np.squeeze(temp_value_sub)

  temp_sub_score_min = np.min(temp_sub_score)
  temp_sub_score_max = np.max(temp_sub_score)

  temp_sub_score_color_norm = (temp_sub_score - temp_sub_score_min) / (temp_sub_score_max - temp_sub_score_min)
  coolwarm_colors = temp_cmap(temp_sub_score_color_norm)

  # find cifti label axis
  cifti_label_axis = copy.deepcopy(cifti_axes[0])
  cifti_brain_model_axis = cifti_axes[1]

  for i_region in range(0, 180):

    temp_orig_tuple = cifti_label_axis[0][1][i_region + 1]
    
    cifti_label_axis[0][1][i_region + 1] = tuple((temp_orig_tuple[0], tuple(coolwarm_colors[i_region])))

  cifti_new_hdr = nib.cifti2.cifti2.Cifti2Header()
  temp_hdr = cifti_new_hdr.from_axes(tuple((cifti_label_axis,cifti_brain_model_axis)))

  # create new image file
  new_img_file = nib.cifti2.cifti2.Cifti2Image(cifti_data, header=temp_hdr, nifti_header=nifti_hdr)

  score_map_path = os.path.join(score_dir_out, i_explained_variance_type + "_" + file_str+'.32k_fs_LR.dlabel.nii')
  new_img_file.to_filename(score_map_path)


In [ ]:
import matplotlib.pyplot as plt 

index_max = num_significant_regions_list.index(max(num_significant_regions_list))
print("index_max:", index_max, "num of regions:", num_significant_regions_list[16])

# Import required packages
from tkinter import font
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pylab import cm
import matplotlib.font_manager as fm

font_names = [f.name for f in fm.fontManager.ttflist]
# print(font_names)

mpl.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 16
plt.rcParams['axes.linewidth'] = 2
fig = plt.figure(figsize=(16, 8))

plt.plot(np.arange(len(num_significant_regions_list)), num_significant_regions_list, 'k-o')  
plt.xticks(np.arange(len(num_significant_regions_list)), num_total_dims_list, rotation ='vertical')

plt.xlabel("total dimensions", fontsize=22)
plt.ylabel("number of regions", fontsize=22)
# plt.title("variance of latent units along depth", fontsize=24)
plt.show()
plt.savefig("results/latent_units_var_depth.eps", dpi=500)
plt.savefig("results/latent_units_var_depth.png", dpi=500)

#### intersection regions of vit midlayer explainable variance score

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy.io import loadmat

project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/spatial_process"
file_dir = project_dir+"/process/shared_region_dims/temp_data/"

temp_cmap = cm.get_cmap('coolwarm')
print(temp_cmap)

p_value_threshold = 0.05

# load all number of dims
from scipy.io import loadmat

file_dir = project_dir+"/process/shared_region_dims/temp_data/"
matdic = loadmat(file_dir + "vit_dims_list_semgeo_shared_region_dims_list_optimal_hard_threshold.mat")
num_total_dims_list = list(np.squeeze(matdic["num_total_dims_array"]))

bool_pvals_list = []
num_significant_regions_list = []
for i_num_total_dims in num_total_dims_list:

  num_total_dims = i_num_total_dims

  file_str = "vit_"+ str(num_total_dims) + "_dims"+ "_midlayer_geometry_ev_score_brain_regions"

  matdic = loadmat(file_dir+file_str+".mat")
  sub_score = np.squeeze(matdic["sub_score"])
  pvals_corrected = np.squeeze(matdic["pvals_corrected"])

  temp_sub_score = copy.deepcopy(sub_score)

  bool_temp = np.array(pvals_corrected < p_value_threshold)

  bool_pvals_list.append(bool_temp)
  print(num_total_dims, "number of effective regions:", np.sum(pvals_corrected < p_value_threshold))

#---filter some dims less than 
intersect_bool_pvals = np.ones_like(bool_pvals_list[0])
for bool_pvals in bool_pvals_list:

  print("current bool-number of effective regions:", np.sum(bool_pvals == True))
  # if np.sum(bool_pvals == True) < 100: continue
  intersect_bool_pvals = np.logical_and(intersect_bool_pvals, bool_pvals)
  print("after and-number of effective regions:", np.sum(intersect_bool_pvals == True))

print("after ALL and-number of effective regions:", np.sum(intersect_bool_pvals == True))

#---save to dlabel
temp_sub_score_min = np.min(temp_sub_score)
temp_sub_score_max = np.max(temp_sub_score)
print("sub_score_min:", temp_sub_score_min)
print("sub_score_max:", temp_sub_score_max)

temp_sub_score[intersect_bool_pvals == True] = temp_sub_score_max
temp_sub_score[intersect_bool_pvals == False] = temp_sub_score_min

temp_sub_score_color_norm = (temp_sub_score - temp_sub_score_min) / (temp_sub_score_max - temp_sub_score_min)
coolwarm_colors = temp_cmap(temp_sub_score_color_norm)

# find cifti label axis
cifti_label_axis = copy.deepcopy(cifti_axes[0])
cifti_brain_model_axis = cifti_axes[1]

for i_region in range(0, 180):

  temp_orig_tuple = cifti_label_axis[0][1][i_region + 1]
  
  cifti_label_axis[0][1][i_region + 1] = tuple((temp_orig_tuple[0], tuple(coolwarm_colors[i_region])))

cifti_new_hdr = nib.cifti2.cifti2.Cifti2Header()
temp_hdr = cifti_new_hdr.from_axes(tuple((cifti_label_axis,cifti_brain_model_axis)))

# create new image file
new_img_file = nib.cifti2.cifti2.Cifti2Image(cifti_data, header=temp_hdr, nifti_header=nifti_hdr)

score_map_path = os.path.join(score_dir_out, 'intersect_vit_dims_midlayer_geometry_ev_score_brain_regions'+'.32k_fs_LR.dlabel.nii')
new_img_file.to_filename(score_map_path)


In [ ]:
# use the dims of max effective regions
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy.io import loadmat

project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/spatial_process"
file_dir = project_dir+"/process/shared_region_dims/temp_data/"

temp_cmap = cm.get_cmap('coolwarm')
print(temp_cmap)

p_value_threshold = 0.05

file_dir = project_dir+"/process/shared_region_dims/temp_data/"
matdic = loadmat(file_dir + "vit_dims_list_semgeo_shared_region_dims_list_optimal_hard_threshold.mat")
num_total_dims_list = list(np.squeeze(matdic["num_total_dims_array"]))

bool_pvals_list = []
for i_num_total_dims in num_total_dims_list:

  num_total_dims = i_num_total_dims
  # num_total_dims = 1094

  file_str = "vit_"+ str(num_total_dims) + "_dims"+ "_midlayer_geometry_ev_score_brain_regions"

  matdic = loadmat(file_dir+file_str+".mat")
  sub_score = np.squeeze(matdic["sub_score"])
  pvals_corrected = np.squeeze(matdic["pvals_corrected"])

  temp_sub_score = copy.deepcopy(sub_score)

  #---save to dlabel
  temp_sub_score_min = np.min(temp_sub_score)
  temp_sub_score_max = np.max(temp_sub_score)
  print("sub_score_min:", temp_sub_score_min)
  print("sub_score_max:", temp_sub_score_max)

  temp_sub_score[pvals_corrected >= p_value_threshold] = np.min(temp_sub_score)

  # temp_sub_score[intersect_bool_pvals == False] = np.min(temp_sub_score)
  # temp_sub_score[intersect_bool_pvals == False] = 0
  # print(num_total_dims, "number of temp_sub_score < 0:", np.sum(temp_sub_score < 0))
  # temp_sub_score[temp_sub_score < 0] = 0

  temp_sub_score_color_norm = (temp_sub_score - np.min(temp_sub_score)) / (np.max(temp_sub_score) - np.min(temp_sub_score))
  coolwarm_colors = temp_cmap(temp_sub_score_color_norm)

  # find cifti label axis
  cifti_label_axis = copy.deepcopy(cifti_axes[0])
  cifti_brain_model_axis = cifti_axes[1]

  for i_region in range(0, 180):

    temp_orig_tuple = cifti_label_axis[0][1][i_region + 1]
    
    cifti_label_axis[0][1][i_region + 1] = tuple((temp_orig_tuple[0], tuple(coolwarm_colors[i_region])))

  cifti_new_hdr = nib.cifti2.cifti2.Cifti2Header()
  temp_hdr = cifti_new_hdr.from_axes(tuple((cifti_label_axis,cifti_brain_model_axis)))

  # create new image file
  new_img_file = nib.cifti2.cifti2.Cifti2Image(cifti_data, header=temp_hdr, nifti_header=nifti_hdr)

  score_map_path = os.path.join(score_dir_out, 'intersect_masked_'+file_str+'.32k_fs_LR.dlabel.nii')
  new_img_file.to_filename(score_map_path)
